In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import *
import keras

from CRNN import KerasCrnn

/home/rytse/anaconda3/envs/ngspc/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/rytse/anaconda3/envs/ngspc/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/rytse/anaconda3/envs/ngspc/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/rytse/anaconda3/envs/ngspc/lib/python3.7/site-

In [2]:
# IMG_HEIGHT = 1026    # number of freq bins (rows / pixels high) in each spectrogram
IMG_HEIGHT = 24
IMG_WIDTH = int(10e3)
N_LABELS = 28
BATCH_SIZE = 64
LSTM_SIZE = 64

In [3]:
def ctc_custom_len(args):
    true_labels, pred_labels, label_lens = args
    pred_len = keras.backend.variable([[pred_labels.shape[1].value]] * BATCH_SIZE)
    return keras.backend.ctc_batch_cost(true_labels,
                                        keras.backend.softmax(pred_labels),
                                        keras.backend.variable(pred_len),
                                        label_lens)

In [4]:
# Construct CRNN architecture
crnn = KerasCrnn(IMG_HEIGHT, N_LABELS, nh=LSTM_SIZE)

# Define tensors for the waterfall inputs, label outputs, and predicted label outputs
true_labels = keras.layers.Input(name='feature_labels', shape=[N_LABELS], dtype='int32')
label_lens = keras.layers.Input(name='label_lens', shape=[1], dtype='int32')
imgs = keras.layers.Input(name='waterfall_input', shape=[IMG_HEIGHT, IMG_WIDTH], dtype='float32')
pred_labels = crnn(imgs)

# Define loss
loss = keras.layers.Lambda(ctc_custom_len, output_shape=(1,), name='ctc')([true_labels, pred_labels, label_lens])

ValueError: strides should be of length 1, 1 or 3 but was 2